In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
import urllib.request
from bs4 import BeautifulSoup
from requests import get
import os
import inspect

In [2]:
options = Options()
options.headless = True
#options.add_argument("window-size=400,600")

driver = webdriver.Firefox(options=options)

url_ = "https://maharerait.mahaonline.gov.in/searchlist/search"

driver.get(url_)

In [3]:
url_ = "https://maharerait.mahaonline.gov.in/searchlist/search"
driver.get(url_)
print(driver.title)

Login Page : Maharashtra Real Estate Regulatory Authority


In [7]:
elems = driver.find_elements_by_class_name("search-pro-details")
print(elems[0].click())

None


In [8]:
sleep(2)
elems = driver.find_element_by_id("Agent")
print(elems.click())

None


In [9]:
elems = driver.find_element_by_id("btnAdvance")
print(elems.click())

None


In [10]:
content = driver.page_source

In [11]:
elems = driver.find_element_by_id("State")
stateDrp = Select(driver.find_element_by_id("State"))
stateDrp.select_by_visible_text("MAHARASHTRA")
#inspect.getmembers(stateDrp._setSelected("MAHARASHTRA"))

In [12]:
distDrp = Select(driver.find_element_by_id("District"))

dist_num = 7
dists = []

dists.append('')
for index, option in enumerate(distDrp.options):
    if(index > 0):
        dists.append(option.get_attribute("innerHTML"))
        print(index, " : " + option.get_attribute("innerHTML"))
        
print(dists)
print(dists[dist_num])

1  : Ahmednagar
2  : Akola
3  : Amravati
4  : Aurangabad
5  : beed
6  : Bhandara
7  : Buldana
8  : Chandrapur
9  : Dhule
10  : Gadchiroli
11  : Gondiya
12  : Hingoli
13  : Jalgaon
14  : Jalna
15  : Kolhapur
16  : Latur
17  : Mumbai City
18  : Mumbai Suburban
19  : Nagpur
20  : Nanded
21  : Nandurbar
22  : Nashik
23  : Osmanabad
24  : Palghar
25  : Parbhani
26  : Pune
27  : Raigarh
28  : Ratnagiri
29  : Sangli
30  : Satara
31  : Sindhudurg
32  : Solapur
33  : Thane
34  : Wardha
35  : Washim
36  : Yavatmal
['', 'Ahmednagar', 'Akola', 'Amravati', 'Aurangabad', 'beed', 'Bhandara', 'Buldana', 'Chandrapur', 'Dhule', 'Gadchiroli', 'Gondiya', 'Hingoli', 'Jalgaon', 'Jalna', 'Kolhapur', 'Latur', 'Mumbai City', 'Mumbai Suburban', 'Nagpur', 'Nanded', 'Nandurbar', 'Nashik', 'Osmanabad', 'Palghar', 'Parbhani', 'Pune', 'Raigarh', 'Ratnagiri', 'Sangli', 'Satara', 'Sindhudurg', 'Solapur', 'Thane', 'Wardha', 'Washim', 'Yavatmal']
Buldana


In [13]:
distDrp.select_by_visible_text(dists[dist_num])

In [14]:
elems = driver.find_element_by_id("btnSearch")
print(elems.click())
sleep(2)

None


In [20]:
with open(dists[dist_num] + '.csv', 'w') as file:
    file.write(
        "Agent Name" + ','
        + "Certificate Number" + ','
        + "Agent Type" + ','
        + "Block Number" + ','
        + "Building Name" + ','
        + "Street Name" + ','
        + "Locality" + ','
        + "Landmark" + ','
        + "State" + ','
        + "Division" + ','
        + "District" + ','
        + "Taluka" + ','
        + "Village" + ','
        + "Pin Code" + ','
        + "Office Number" + ','
        + "Website URL" + ','
        + '\n'
    )
    
    find_agents(driver.page_source)

0  : Information Type
1  : Other Than Individual
2  : 
3  : Name
4  : DREAM VISION ADVISORS PVT. LTD.
5  : Organization Type
6  : Company
7  : Description For Other Type Organization
8  : NA
9  : Do you have any Past Experience ?
10  : Yes
11  : Block Number
12  : 1
13  : Building Name
14  : YOGIRAJ APARTMENTS
15  : Street Name
16  : OPP BDCC BANK
17  : Locality
18  : BHONDE SARKAR CHOWK
19  : Land mark
20  : BHONDE SARKAR CHOWK
21  : State/UT
22  : MAHARASHTRA
23  : Division
24  : Amravati
25  : District
26  : Buldana
27  : Taluka
28  : Buldana
29  : Village
30  : buldana (MCL)
31  : Pin Code
32  : 443001
33  : Office Number
34  : 07262247201
35  : Website URL
36  : 
37  : 
38  : Aadhar No. :
39  : 
40  : House No. :
41  : 
42  : Building :
43  : 
44  : Street :
45  : 
46  : Locality :
47  : 
48  : LandMark :
49  : 
50  : State/UT :
51  : 
52  : Division :
53  : 
54  : District :
55  : 
56  : Taluka :
57  : 
58  : Village :
59  : 
60  : Pincode :
61  : 
Completed


In [16]:
def find_agents(html):
    soup = BeautifulSoup(html, "html.parser")
    rows = soup.find_all("tr", class_='grid-row')
    for agent_row in rows:
        cert = agent_row.find_all("td", class_='grid-cell')[2].text
        #print(cert)
        link = agent_row.find_all("td", class_='grid-cell')[3].find_all("a")[0].attrs['href']
        #print(link.find_all("a")[0].attrs['href'])
        Agent(link, cert)
    print('Completed')

In [19]:
def Agent(link, cert):
    global file
    url_ = 'https://maharerait.mahaonline.gov.in' + link
    r = get(url_)

    html = BeautifulSoup(r.content, "html.parser")
    datas = html.find_all("div", class_='col-md-3')
    
    if (datas[1].text.strip() != 'Individual'):
        for index, data in enumerate(datas):
            print(index, " : " + data.text.strip())
    
    file.write(datas[4].text.strip() + ' ' + datas[6].text.strip() + ' ' + datas[8].text.strip() + ','
               + cert + ','
               '"' + datas[1].text.strip() + '",'
               '"' + datas[18].text.strip() + '",'
               '"' + datas[20].text.strip() + '",'
               '"' + datas[22].text.strip() + '",'
               '"' + datas[24].text.strip() + '",'
               '"' + datas[26].text.strip() + '",'
               '"' + datas[28].text.strip() + '",'
               '"' + datas[30].text.strip() + '",'
               '"' + datas[32].text.strip() + '",'
               '"' + datas[34].text.strip() + '",'
               '"' + datas[36].text.strip() + '",'
               '"' + datas[38].text.strip() + '",'
               '"' + datas[40].text.strip() + '",'
               '"' + datas[42].text.strip() + '",'
               + '\n'
              )
    
    #for index, data in enumerate(datas):
        #print(index, " : " + data.text.strip())